# Using Google Cloud Vision API to Retrieve Photo Information
This is a document that outlines steps to using Google's Vision AI API (https://cloud.google.com/vision) for local photos.

## Requirements
- A Google Cloud account in the Free Tier. See https://cloud.google.com/free/docs/gcp-free-tier
- Billing Information (Google claims it will not auto charge to card if doing a free trial. The free trial lasts until $300 credit is used up or until a year has past since sign up).
- Python. Supported versions are 3.5 to 3.7, and 2.7.9 or higher. Recommend >= 3.6 
- Linux, macOS, or Windows device 
Cloud SDK requires Python. Some tools bundled with Cloud SDK have additional requirements


## Creating Google Cloud account
1) Create Google Cloud information by creating a Google Cloud account "Get Started for Free" or "Sign In" through your Google account. See here for instructions from Google. https://console.cloud.google.com/getting-started/checklist

![photo](1.png)

2) Install Google Cloud SDK 
- Windows: https://cloud.google.com/sdk/docs/downloads-interactive#windows
- Mac: https://cloud.google.com/sdk/docs/downloads-interactive#mac
- Linux: https://cloud.google.com/sdk/docs/downloads-interactive#linux

3) Enable API through https://console.cloud.google.com/ by clicking on the 3 bars icon > "API & Services" which will open to a new webpage.

![photo](2.png)

4) Click on "+ Enable APIs and Services" and search for "Cloud Vision API"

![photo](3.png)![photo](4.png)

5) Click the blue "Enable API" button and select Google Cloud Project you want the API enabled (The screenshot shows up as "Manage" because it is already enabled")

![photo](5.png)

6) Install the Cloud Vision API (this is different from Cloud SDK). Please see https://cloud.google.com/vision/docs/setup

## Using Google Cloud Vision API with Python
### Example Use Case: Identifying overhead street name sign text
Before beginning you have to installl Google Cloud SDK, enable Google Cloud Vision API, Create a Project, Enable Billing, <b>and</b> retrieve your personal google application credentials in order to use the API. Keep it secret. 

See https://cloud.google.com/sdk/install as it explains the steps for installation in detail.

Some packages to import are: `io`, `os`, `google.cloud`, and `google.cloud.vision`

Recommended to import `pandas` for data wrangling and `xlsx`/`openpyxl` to export results to excel file.

In [11]:
import io
import os
import pandas as pd
from google.cloud import vision
from google.cloud.vision import types

Put in your google app credentials. You can download the JSON file here https://console.cloud.google.com/apis/credentials/serviceaccountkey when signed in. Keep this file secret.

In [12]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= "/Users/SusanneGov/Documents/My First Project-bcfca3ee21b0.json" # replace with your JSON path
client = vision.ImageAnnotatorClient()

To grab information from a collection of photos.

In [13]:
photo_folder = os.getcwd() + r'/test_photos' # or use your own folder path see comment below
# photo_folder = r"/path/sub_path/folder"
print("The file path is " + photo_folder)

The file path is /Users/SusanneGov/Documents/Python Projects/Signs-and-Markings-Projects/Guide/test_photos


Create dictionaries of text, label, and object detection.

In [14]:
text_info = {} # to put in text annotations
label_info = {} # to put in labels
object_info = {} # to put in multiple objects

Will go through the photo folder collection and read text, label, and objects that have a .png extention.

In [15]:
# goes through photo folder and identifies texts, labels, and objects
for dirpath, subdirs, files in os.walk(photo_folder):
    for x in files:
        if x.endswith(".png"): # can be jpeg
            photo_path = os.path.join(dirpath, x)
            with io.open(photo_path, 'rb') as image_file:
                content = image_file.read()
            image = types.Image(content=content) 
            texts = client.text_detection(image=image).text_annotations
            labels = client.label_detection(image=image).label_annotations
            objects = client.object_localization(image=image).localized_object_annotations
            object_info[x] = [object_.name for object_ in objects] # creates list of objects identified
            label_info[x] =  [label.description for label in labels] # creates list of labels identified
            text_info[x] =  str(texts[0].description).split("\n") # creates list of texts identified

Create pandas dataframe to display results.

In [16]:
texts = {}
labels = {}
objects = {}
for keys in text_info:
    texts[keys] = [x.upper() for x in text_info[keys]]
    labels[keys] = [x.upper() for x in label_info[keys]]
    objects[keys] = [x.upper() for x in object_info[keys]]
texts = pd.DataFrame.from_dict(texts,orient='index').sort_index()
labels = pd.DataFrame.from_dict(labels,orient='index').sort_index()
objects = pd.DataFrame.from_dict(objects,orient='index').sort_index()

Export results into excel file

In [17]:
output = os.getcwd() + r'/example_output.xlsx'
with pd.ExcelWriter(output) as writer:  
    texts.to_excel(writer, sheet_name='texts')
    labels.to_excel(writer, sheet_name='labels')
    objects.to_excel(writer, sheet_name='objects')
    writer.save()
    writer.close()

The results are in example_output.xlsx 

- Test 1 Photo ![photo](test_photos/test1.png)

- Test 2 Photo ![photo](test_photos/test2.png)

In [18]:
display(texts)

,0,1,2,3,4,5,6,7,8,9,10
test1-checkpoint.png,CONNECTED,PHARMACY CARE,GOES BEYOND,PRESCRIPTIONS.,LEFT TURN,YIELD,ON GREEN,LAMAR,- ENFIELD RD,PARKWAY 1200,
test1.png,CONNECTED,PHARMACY CARE,GOES BEYOND,PRESCRIPTIONS.,LEFT TURN,YIELD,ON GREEN,LAMAR,- ENFIELD RD,PARKWAY 1200,
test2-checkpoint.png,PARKWAY,RD,ENFIELD,1200,LAMAR,STHE T VERN,TAVERN,"OPEN ""GLASS",CONTAINEPS,PROHUBIT,
test2.png,PARKWAY,RD,ENFIELD,1200,LAMAR,STHE T VERN,TAVERN,"OPEN ""GLASS",CONTAINEPS,PROHUBIT,


For test1.png, the vision cloud api identified overhead street name sign texts on columns 8 and 9.

For test2.png,the vision cloud api identified overhead street name sign texts on columns 0 to 4.

In [19]:
display(labels)

,0,1,2,3,4,5,6,7,8,9
test1-checkpoint.png,ROAD,TRAFFIC LIGHT,SIGNALING DEVICE,LANE,LIGHTING,TRANSPORT,MOTOR VEHICLE,THOROUGHFARE,HIGHWAY,LIGHT FIXTURE
test1.png,ROAD,TRAFFIC LIGHT,SIGNALING DEVICE,LANE,LIGHTING,TRANSPORT,MOTOR VEHICLE,THOROUGHFARE,HIGHWAY,LIGHT FIXTURE
test2-checkpoint.png,TRAFFIC LIGHT,SIGNALING DEVICE,LIGHTING,LIGHT FIXTURE,URBAN AREA,SIGNAGE,ROAD,STREET LIGHT,TREE,SIGN
test2.png,TRAFFIC LIGHT,SIGNALING DEVICE,LIGHTING,LIGHT FIXTURE,URBAN AREA,SIGNAGE,ROAD,STREET LIGHT,TREE,SIGN


For test1.png, the vision cloud api did not identify signage.

For test2.png,the vision cloud api identified signage.

In [20]:
display(objects)

,0,1
test1-checkpoint.png,CAR,CAR
test1.png,CAR,CAR
test2-checkpoint.png,LIGHTING,STREET LIGHT
test2.png,LIGHTING,STREET LIGHT


For test1.png, the vision cloud api only identified cars

For test2.png,the vision cloud api only identified lighting and street lights.

The output comes with bounding box geometries (texts/labels annotations) and a confidence score (labels/objects annotations). You can create your own object detector with Google's AutoML without having to know machine learning modelling.